# Coloring Algorithm
Final code / workflow for the coloring algorithm.

In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
import sys

sys.path.append('../')
# from algorithm.Voxel import Voxel
from algorithm.Lattice import Lattice
# from algorithm.Surroundings import Surroundings
# from algorithm.SymmetryDf import SymmetryDf
# from algorithm.BondPainter import Mesovoxel, BondPainter
from algorithm.Painter import Painter
from algorithm.Relation import Relation
from algorithm.ColorTree import ColorTree

from app.design.Designer import RunDesigner
from app.visualize.Visualizer import RunVisualizer

import numpy as np
import pandas as pd
from PyQt6.QtWidgets import QApplication
from PyQt6.QtCore import QCoreApplication

## The algorithm

### Create the desired lattice structure

Saves it to a file `data/lattice2.npy` to be loaded in a separate cell. Alternatively, you can define your own numpy array in code and create the lattice that way.

In [23]:
%gui qt

from app.design.Designer import RunDesigner

if __name__ == '__main__':
    if not QCoreApplication.instance():
        app = QApplication(sys.argv)
    else:
        app = QCoreApplication.instance()
    
    designer = RunDesigner(app)
    input_lattice = designer.run()
    
    if input_lattice is not None:
        print(f'Lattice received.')
        np.save("data/lattice.npy", input_lattice)
    else:
        print("No lattice received.")

Saved lattice:
[[[1 1]
  [1 1]]

 [[0 0]
  [0 0]]

 [[0 0]
  [0 0]]]

Lattice received.


### Run the algorithm

1. Creates lattice structure with voxel, bond objects.
2. Create the full surroundings matrix which is used for comparing symmetry operations on a tiled lattice structure.
3. Paint the bond with the algorithm

In [14]:
input_lattice = np.load('data/double_oreo.npy')
# input_lattice = np.load('data/lattice.npy')

lattice = Lattice(input_lattice)
lattice.compute_symmetries()

# Paint the lattice
painter = Painter(lattice)
painter.str_paint_lattice()
painter.comp_paint_lattice()

# # Print
print(f"Final structural voxels: {painter.mesovoxel.structural_voxels}")
print(f"Final complementary voxels: {painter.mesovoxel.complementary_voxels}")

# Bond painter stuff
# bond_painter = BondPainter(lattice)
# bond_painter.paint_lattice()

Paint bond with color (1):
Between voxel 0 ((0, 0, -1)) and voxel 4 ((0, 0, 1))

Painting self-symmetries for newly painted structural voxels...
Paint bond with color (2):
Between voxel 0 ((1, 0, 0)) and voxel 1 ((-1, 0, 0))

Child voxel 1 and parent voxel 0 satisfy loose negation
Voxel 1 has best_mesoparent voxel 0, sym_label translation, flip_complementarity is True
Adding voxel 1 to complementary set.
Child voxel 2 is partner with parent voxel 1? False
Child voxel 2 and parent voxel 1 satisfy loose equality
Voxel 2 has best_mesoparent voxel 1, sym_label translation, flip_complementarity is False
Child voxel 3 is partner with parent voxel 1? True
Child voxel 3 and parent voxel 1 satisfy loose negation
Voxel 3 has best_mesoparent voxel 1, sym_label translation, flip_complementarity is True
Child voxel 6 is partner with parent voxel 1? False
Child voxel 6 and parent voxel 4 satisfy loose negation
Voxel 6 has best_mesoparent voxel 4, sym_label translation, flip_complementarity is True
C

In [16]:
from algorithm.BindingFlexibility import BindingFlexibility
# print(lattice.get_voxel(0).color_dict())

bf = BindingFlexibility(lattice)
print(bf.get_sympartners(4))

bf1_lattice = bf.binding_flexibility_1()
# bf3_lattice = bf.binding_flexibility_3(max_cutoff_ratio=1/6)

[['+x', '-y', '+y', '-z'], ['+y', '-y', '-z', '-x']]


## Minimum origami
Once a valid coloring scheme is found, we find the complementary combination which produces the minimum unique number of origami.

In [15]:
# Compute the minimum number of origami via game tree search
# all_color_configs = lattice.init_all_color_configs()
# for color in all_color_configs:
#     print(f'Color {color} has {len(all_color_configs[color])} configurations.')

color_tree = ColorTree(lattice)
lattice.reset_color_config()
print("Initial unique origami:", lattice.unique_origami())
# reduced_color_configs = color_tree._reduce_color_configs(color_tree.all_color_configs)

optimal_path = color_tree.optimize(end_early=False)
print("Optimal path of color configurations:", optimal_path)

# # # Apply the optimal color_tree path to the lattice
# lattice.apply_color_configs(optimal_path)
print(f'Final unique origami: {lattice.unique_origami()}')

# color_tree.print_optimal_path(optimal_path)

Initial unique origami: [0, 1, 4, 5, 11, 15]
Evaluating color-wise configurations for 5 colors...
Evaluating color 5/5, config 16/16.....
Done with color-wise evaluation.
Color 1 has 4 configurations.
Color 2 has 2 configurations.
Color 3 has 2 configurations.
Color 4 has 2 configurations.
Color 5 has 2 configurations.
Searching for minimum origami across 64 possibilities...
Evaluating color-wise configurations for 5 colors...
Evaluating color 5/5, config 16/16.....
Done with color-wise evaluation.
Evaluating color-wise configurations for 5 colors...
Evaluating color 5/5, config 2/2...
Done with color-wise evaluation.
Done! 4 minimum unique origami found.
Optimal path of color configurations: {1: {0: 1, 1: -1, 2: -1, 3: 1, 4: -1, 5: 1, 6: 1, 7: -1, 8: 1, 9: -1, 10: -1, 11: 1, 12: -1, 13: 1, 14: 1, 15: -1}, 2: {0: 1, 1: -1, 2: -1, 3: 1, 12: -1, 13: 1, 14: 1, 15: -1}, 3: {0: 1, 1: -1, 2: -1, 3: 1, 12: -1, 13: 1, 14: 1, 15: -1}, 4: {4: 1, 5: -1, 6: -1, 7: 1, 8: -1, 9: 1, 10: 1, 11: -1}, 5

In [ ]:
for color, configs in reduced_color_configs.items():
    print(f'Color {color} has {len(configs)} configurations.')
    for config in configs:
        lattice.reset_color_config()
        lattice.apply_color_configs({color: config})
        unique_origami = lattice.unique_origami()
        print(f'{len(unique_origami)} Unique origami: {unique_origami}')

In [65]:
for color, voxels in lattice.init_colordict().items():
    print(f'Color {color}: {voxels}')

Color 1: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Color 2: [0, 1, 2, 3, 12, 13, 14, 15]
Color 3: [0, 1, 2, 3, 12, 13, 14, 15]
Color 4: [4, 5, 6, 7, 8, 9, 10, 11]
Color 5: [4, 5, 6, 7, 8, 9, 10, 11]


In [128]:
# Lists to store combined voxel and bond information
final_df = lattice.final_df(show_bond_type=False)
final_df

Voxel                      Bond Colors               
     ID Material Coordinates          +x -x +y -y +z -z
0     0        1   (0, 1, 1)           1  1  1  1  1  1
1     1        0   (1, 1, 1)          -1 -1  2  2  2  2
2     2        0   (0, 0, 1)           2  2 -1 -1  2  2
3     3        2   (1, 0, 1)          -2 -2 -2 -2  3  3
4     4        0   (0, 1, 0)           2  2  2  2 -1 -1
5     5        2   (1, 1, 0)          -2 -2  3  3 -2 -2
6     6        2   (0, 0, 0)           3  3 -2 -2 -2 -2
7     7        3   (1, 0, 0)          -3 -3 -3 -3 -3 -3

In [30]:
final_df = lattice.final_df(show_bond_type=True)
final_df

Voxel                         Bond Colors                                \
      ID Material Coordinates             +x             -x             +y   
0      0        1   (0, 1, 3)  complementary  complementary  complementary   
1      1        1   (1, 1, 3)  complementary  complementary  complementary   
2      2        1   (0, 0, 3)  complementary  complementary  complementary   
3      3        1   (1, 0, 3)  complementary  complementary  complementary   
4      4        0   (0, 1, 2)  complementary  complementary  complementary   
5      5        0   (1, 1, 2)  complementary  complementary  complementary   
6      6        0   (0, 0, 2)  complementary  complementary  complementary   
7      7        0   (1, 0, 2)  complementary  complementary  complementary   
8      8        0   (0, 1, 1)  complementary  complementary  complementary   
9      9        0   (1, 1, 1)  complementary  complementary  complementary   
10    10        0   (0, 0, 1)  complementary  complementary  complementary   
11    11        0   (1, 0, 1)  complementary  complementary  complementary   
12    12        1   (0, 1, 0)  complementary  complementary  complementary   
13    13        1   (1, 1, 0)  complementary  complementary  complementary   
14    14        1   (0, 0, 0)  complementary  complementary  complementary   
15    15        1   (1, 0, 0)  complementary  complementary  complementary   

                                                 
               -y             +z             -z  
0   complementary  complementary     structural  
1   complementary  complementary     structural  
2   complementary  complementary     structural  
3   complementary  complementary     structural  
4   complementary     structural  complementary  
5   complementary     structural  complementary  
6   complementary     structural  complementary  
7   complementary     structural  complementary  
8   complementary  complementary     structural  
9   complementary  complementary     structural  
10  complementary  complementary     structural  
11  complementary  complementary     structural  
12  complementary     structural  complementary  
13  complementary     structural  complementary  
14  complementary     structural  complementary  
15  complementary     structural  complementary

### View the final colored lattice

In [15]:
%gui qt

if __name__ == '__main__':
    if not QCoreApplication.instance():
        app = QApplication(sys.argv)
    else:
        app = QCoreApplication.instance()

    visualizeWindow = RunVisualizer(lattice, app)

In [13]:
print(len(lattice.unique_origami()))
print(bond_painter.mesovoxel.structural_voxels)

4
{0, 4}


In [5]:
symlist = lattice.symmetry_df.symlist(0, 12)
symlist

['180° X-axis',
 '180° Y-axis',
 '180° Z-axis + 180° X-axis',
 '180° Z-axis + 180° Y-axis',
 '270° Z-axis + 180° X-axis',
 '270° Z-axis + 180° Y-axis',
 '90° Z-axis + 180° X-axis',
 '90° Z-axis + 180° Y-axis']

In [9]:
df = lattice.symmetry_df.symmetry_df
df

,translation,90° X-axis,180° X-axis,270° X-axis,90° Y-axis,180° Y-axis,270° Y-axis,90° Z-axis,180° Z-axis,270° Z-axis,...,270° Y-axis + 90° X-axis,270° Y-axis + 90° Z-axis,270° Z-axis + 180° Y-axis,270° Z-axis + 90° X-axis,90° X-axis + 180° Y-axis,90° Y-axis + 270° Z-axis,90° Y-axis + 90° X-axis,90° Z-axis + 180° Y-axis,90° Z-axis + 90° X-axis,90° Z-axis + 90° Y-axis
"(2, 8)",False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
"(2, 3)",True,False,False,False,False,False,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False
"(10, 13)",False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
"(7, 8)",False,False,True,False,False,True,False,False,False,False,...,False,False,True,False,False,False,False,True,False,False
"(7, 14)",False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(5, 14)",False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
"(12, 15)",True,False,False,False,False,False,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False
"(3, 10)",False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
(7),True,False,False,False,False,False,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False


In [13]:
# Relation tests

# for i in range(0, 16):
#     for j in range (i+1, 16):
#         voxel1 = lattice.get_voxel(i)
#         voxel2 = lattice.get_voxel(j)

#         rel = Relation.get_voxel_relation(voxel1, voxel2)
#         print(f"Voxel {voxel1.id} and voxel {voxel2.id} have {rel} relation.")


voxel1 = lattice.get_voxel(0)
voxel2 = lattice.get_voxel(1)
rel = Relation.get_voxel_relation(voxel1, voxel2)
print(f"Voxel {voxel1.id} and voxel {voxel2.id} have {rel} relation.")

Voxel 0 and voxel 1 have loose equality relation.


## Flip Complementarity Tests
We can also manually flip the complementarity. The following code produces the minimum result when applied to the perovskite.

In [69]:
voxel = lattice.get_voxel(8)
print(voxel.flip_complementarity(color=3))

voxel = lattice.get_voxel(4)
print(voxel.flip_complementarity(color=4))

voxel = lattice.get_voxel(7)
print(voxel.flip_complementarity(color=4))

# voxel.print_bonds()

[<algorithm.Voxel.Voxel object at 0x16d905340>, <algorithm.Voxel.Voxel object at 0x16d907cb0>, <algorithm.Voxel.Voxel object at 0x16d8d0b00>, <algorithm.Voxel.Voxel object at 0x16d8d1910>]
[<algorithm.Voxel.Voxel object at 0x15f3d1610>, <algorithm.Voxel.Voxel object at 0x15f3d2510>]
[<algorithm.Voxel.Voxel object at 0x16d823a40>, <algorithm.Voxel.Voxel object at 0x2889c4350>]


## Test post-processing speed

Found that deepcopy() created much of the time issues.

In [47]:
import cProfile
import pstats
import io
from pstats import SortKey

def profile_function(func, *args, **kwargs):
    pr = cProfile.Profile()
    pr.enable()
    result = func(*args, **kwargs)  # Run the function you want to profile
    pr.disable()

    s = io.StringIO()
    ps = pstats.Stats(pr, stream=s).sort_stats(SortKey.CUMULATIVE)
    ps.print_stats()
    print(s.getvalue())
    
    return result  # Return the result of the function

# Profile lattice.unique_origami()
profile_function(lattice.unique_origami)

# Profile lattice.apply_color_configs(optimal_path)
# profile_function(lattice.apply_color_configs, optimal_path)

         24441 function calls (24200 primitive calls) in 0.020 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.017    0.017 /Users/sarah/Desktop/ganglab/origami/origami-venv/lib/python3.12/site-packages/decorator.py:229(fun)
        1    0.000    0.000    0.017    0.017 /Users/sarah/Desktop/ganglab/origami/origami-venv/lib/python3.12/site-packages/IPython/core/history.py:55(only_when_enabled)
      2/1    0.000    0.000    0.017    0.017 /Users/sarah/Desktop/ganglab/origami/notebooks/../algorithm/Lattice.py:136(unique_origami)
       60    0.001    0.000    0.011    0.000 /Users/sarah/Desktop/ganglab/origami/notebooks/../algorithm/Rotation.py:178(rotate_voxel_bonds)
       30    0.000    0.000    0.008    0.000 /Users/sarah/Desktop/ganglab/origami/notebooks/../algorithm/SymmetryDf.py:50(symlist)
      168    0.000    0.000    0.005    0.000 /Users/sarah/Desktop/ganglab/origami/notebooks/..

[0, 1, 4, 5]

## The great Relation Refactoring

In [22]:
%gui qt

from app.design.Designer import RunDesigner

if __name__ == '__main__':
    if not QCoreApplication.instance():
        app = QApplication(sys.argv)
    else:
        app = QCoreApplication.instance()
    
    designer = RunDesigner(app)
    input_lattice = designer.run()
    
    if input_lattice is not None:
        print(f'Lattice received.')
        np.save("data/lattice.npy", input_lattice)
    else:
        print("No lattice received.")

Saved lattice:
[[[1 1]
  [1 1]]

 [[0 0]
  [0 0]]

 [[2 2]
  [2 2]]

 [[0 0]
  [0 0]]]

Lattice received.


In [23]:
import sys
sys.path.append('../')

from algorithm.Mesovoxel import Mesovoxel
from algorithm.Painter2 import Painter
from algorithm.Lattice import Lattice
import numpy as np

input_lattice = np.load('data/lattice.npy')
# input_lattice = np.load('data/lattice.npy')

lattice = Lattice(input_lattice)
lattice.compute_symmetries()

# mesovoxel = Mesovoxel(lattice)
painter = Painter(lattice)
painter.str_paint_lattice()
painter.comp_paint_lattice()

	 ---> painting 1 onto voxel 0's bond -z with type structural 
	 ---> painting -1 onto voxel 4's bond +z with type structural 
Paint new s_bond (1):
Between voxel 0 ((0, 0, -1)) and voxel 4 ((0, 0, 1))

	 ---> painting 1 onto voxel 0's bond +z with type structural 
	 ---> painting -1 onto voxel 12's bond -z with type structural 
	 ---> painting 2 onto voxel 8's bond +z with type structural 
	 ---> painting -2 onto voxel 4's bond -z with type structural 
Paint new s_bond (2):
Between voxel 8 ((0, 0, 1)) and voxel 4 ((0, 0, -1))

	 ---> painting 2 onto voxel 8's bond -z with type structural 
	 ---> painting -2 onto voxel 12's bond +z with type structural 
	 ---> painting 3 onto voxel 0's bond +x with type complementary 
	 ---> painting -3 onto voxel 1's bond -x with type complementary 
Paint new c_bond (3):
Between voxel 0 ((1, 0, 0)) and voxel 1 ((-1, 0, 0))

	 ---> painting 3 onto voxel 0's bond -x with type complementary 
	 ---> painting -3 onto voxel 1's bond +x with type complementa

In [24]:
%gui qt
from PyQt6.QtWidgets import QApplication
from PyQt6.QtCore import QCoreApplication

from app.visualize.Visualizer import RunVisualizer

if __name__ == '__main__':
    if not QCoreApplication.instance():
        app = QApplication(sys.argv)
    else:
        app = QCoreApplication.instance()

    visualizeWindow = RunVisualizer(lattice, app)

In [16]:
unique_origami = lattice.unique_origami()
print(f"found {len(unique_origami)} unique origami")
print(unique_origami)

found 27 unique origami
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]


In [18]:
for mvoxel in painter.mesovoxel.mvoxels:
    print(mvoxel)

MVoxel(id=0, type=structural, mesopartner=None, maplist=[0])
MVoxel(id=1, type=structural, mesopartner=None, maplist=[1])
MVoxel(id=2, type=structural, mesopartner=None, maplist=[2])
MVoxel(id=3, type=structural, mesopartner=15, maplist=[3])
MVoxel(id=4, type=structural, mesopartner=16, maplist=[4])
MVoxel(id=5, type=structural, mesopartner=17, maplist=[5])
MVoxel(id=6, type=structural, mesopartner=None, maplist=[6])
MVoxel(id=7, type=structural, mesopartner=None, maplist=[7])
MVoxel(id=8, type=structural, mesopartner=None, maplist=[8])
MVoxel(id=9, type=structural, mesopartner=18, maplist=[9])
MVoxel(id=10, type=structural, mesopartner=19, maplist=[10])
MVoxel(id=11, type=structural, mesopartner=20, maplist=[11])
MVoxel(id=12, type=structural, mesopartner=None, maplist=[12])
MVoxel(id=13, type=structural, mesopartner=None, maplist=[13])
MVoxel(id=14, type=structural, mesopartner=None, maplist=[14])
MVoxel(id=15, type=complementary, mesopartner=3, maplist=[21])
MVoxel(id=16, type=compl

In [4]:
df = lattice.final_df()
df

Voxel                      Bond Colors               
      ID Material Coordinates          +x -x +y -y +z -z
0      0        1   (0, 1, 3)           2  2  2  2  3  1
1      1        1   (1, 1, 3)          -2 -2 -2 -2 -3  1
2      2        1   (0, 0, 3)          -2 -2 -2 -2 -3  1
3      3        1   (1, 0, 3)           2  2  2  2  3  1
4      4        0   (0, 1, 2)           4  4  4  4 -1  5
5      5        0   (1, 1, 2)          -4 -4 -4 -4 -1 -5
6      6        0   (0, 0, 2)          -4 -4 -4 -4 -1 -5
7      7        0   (1, 0, 2)           4  4  4  4 -1  5
8      8        0   (0, 1, 1)          -4 -4 -4 -4 -5 -1
9      9        0   (1, 1, 1)           4  4  4  4  5 -1
10    10        0   (0, 0, 1)           4  4  4  4  5 -1
11    11        0   (1, 0, 1)          -4 -4 -4 -4 -5 -1
12    12        1   (0, 1, 0)          -2 -2 -2 -2  1 -3
13    13        1   (1, 1, 0)           2  2  2  2  1  3
14    14        1   (0, 0, 0)           2  2  2  2  1  3
15    15        1   (1, 0, 0)          -2 -2 -2 -2  1 -3